<a href="https://colab.research.google.com/github/zIanHxT/axcf596843/blob/main/DFL-Colab-Shraeyas.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

---
###**Test Your GPU**
---
Run following command to check if your **GPU** is working.

In [2]:
!nvidia-smi

Wed Dec 29 13:59:09 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 495.44       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla K80           Off  | 00000000:00:04.0 Off |                    0 |
| N/A   33C    P8    29W / 149W |      0MiB / 11441MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

##**In case, you get an error for the above code** :
If you get this error **"NVIDIA-SMI has failed because it couldn't communicate with the NVIDIA driver. Make sure that the latest NVIDIA driver is installed and running."**

Go to **"Runtime"** from the menu bar and then click on **"Change Runtime Type"**, Select **GPU** as the Hardware Accelerator and click save.
Then try and **run the above command again.**

If it still doesn't work, that means **Google Colab has limited your resources** for now. The resources generally get refreshed after **12 hours**

####**Not to mention, you can always try with a different google account.**

---
###**Execute following code to prevent Random Disconnects**
---

In [ ]:
import IPython
display(IPython.display.Javascript('''
  function reconnect()
  {
    console.log("Script Working");
    btn = document.querySelector("colab-connect-button");
    if (btn != null)
    {
      console.log("Working Clicking Reconnect Button 1");
      btn.click();
    }
    btn = document.getElementById('ok');
    if (btn != null)
    {
      console.log("Working Clicking Reconnect Button 1");
      btn.click();
    }
  }
  setInterval(reconnect, 5000);
'''))

print ("Finished")

------------------------------------------------------------------
#**Prepare Your Data**
------------------------------------------------------------------

Mount Your **Google Drive** by executing the following code.

In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)


In [ ]:
%cd "drive"
%cd "My Drive"

Clone the Source Code from **Github** and Install **Requirements** using **PIP**

In [ ]:
!git clone https://github.com/Shraeyas/DFL-Colab-Shraeyas 
%cd DFL-Colab-Shraeyas
!pip install -r requirements-colab.txt
!echo "Done."

------------------------------------------------------------------
##If you clicked on **RESTART RUNTIME** in the above step, then execute the following lines.
------------------------------------------------------------------

In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)
%cd "drive"
%cd "My Drive"
%cd "DFL-Colab-Shraeyas"
!pip install -r requirements-colab.txt

------------------------------------------------------------------
#**Add your data_dst and data_src video files**
------------------------------------------------------------------
Open Your Google Drive.

The Above Step will create a Folder Called **DFL-Colab-Shraeyas** on your **Google Drive**.

Inside that folder, there is a **workspace** folder.

You need to place your **data_dst.mp4** and **data_src.mp4** inside that folder. To do this, simply **drag and drop** these files from your PC to this folder.

After you put those files in Google Drive, the contents of the workspace folder should look like this :

      workspace
        |- data_dst
        |- data_src
        |- model
        |- data_dst.mp4
        |- data_src.mp4


------------------------------------------------------------------
#**Extract Frames From Video**
------------------------------------------------------------------


Extract Frames From **data_dst**

In [ ]:
!python main.py videoed extract-video --input-file workspace/data_dst.* --output-dir workspace/data_dst/

Extract Frames From **data_src**

In [ ]:
!python main.py videoed extract-video --input-file workspace/data_src.* --output-dir workspace/data_src/

------------------------------------------------------------------
#**Denoise Faces**
------------------------------------------------------------------


Denoise Frames From **data_dst**

In [ ]:
!python main.py videoed denoise-image-sequence --input-dir workspace/data_dst

Denoise Frames From **data_src**

In [ ]:
!python main.py videoed denoise-image-sequence --input-dir workspace/data_src

------------------------------------------------------------------
#**Detect Faces and Align**
------------------------------------------------------------------


In [ ]:
#@markdown Select the Type of **Detector** (s3fd recommended) and **FaceType**
Detector = "s3fd" #@param ["s3fd"]
FaceType = "whole_face" #@param ["head", "half_face", "full_face", "whole_face"]

In [ ]:
#@markdown ###For **data_dst**
#@markdown <sup>**!python** main.py **extract** --input-dir workspace/data_dst --output-dir workspace/data_dst/aligned --face-type " + $FaceType + " --detector " + $Detector</sup>

cmd = "main.py extract --input-dir workspace/data_dst --output-dir workspace/data_dst/aligned --face-type " + FaceType + " --detector " + Detector
!python $cmd

In [ ]:
#@markdown ###For **data_src**
#@markdown <sup>**!python** main.py **extract** --input-dir workspace/data_src --output-dir workspace/data_src/aligned --face-type " + $FaceType + " --detector " + $Detector</sup>

cmd = "main.py extract --input-dir workspace/data_src --output-dir workspace/data_src/aligned --face-type " + FaceType + " --detector " + Detector
!python $cmd

------------------------------------------------------------------
#**Sort Aligned Faces**
------------------------------------------------------------------


In [ ]:
#@markdown Select **sort type** (absdiff recommended)
sort_type = "absdiff" #@param ["absdiff", "blur", "face-yaw", "face-pitch", "face-source-rect-size", "hist", "hist-dissim", "brightness", "hue", "black", "origname", "oneface", "final", "final-faster"]

In [ ]:
#@markdown For **data_dst**

#@markdown <sup>**!python** main.py **sort** --input-dir workspace/data_dst/aligned --by " + $sort_type

cmd = "main.py sort --input-dir workspace/data_dst/aligned --by "+sort_type
!python $cmd

In [ ]:
#@markdown For **data_src**

#@markdown <sup>**!python** main.py **sort** --input-dir workspace/data_src/aligned --by " + $sort_type

cmd = "main.py sort --input-dir workspace/data_src/aligned --by "+sort_type
!python $cmd

------------------------------------------------------------------
#**Faceset Enhance**
------------------------------------------------------------------

In [ ]:
#@markdown For **data_dst**

#@markdown <sup>**!python** main.py **facesettool** enhance --input-dir /workspace/data_dst/aligned</sup>
!python main.py facesettool enhance --input-dir /workspace/data_dst/aligned

In [ ]:
#@markdown For **data_src**

#@markdown <sup>**!python** main.py **facesettool** enhance --input-dir /workspace/data_src/aligned</sup>
!python main.py facesettool enhance --input-dir /workspace/data_src/aligned

------------------------------------------------------------------
#**Train Model**
------------------------------------------------------------------

In [ ]:
Model = "SAEHD" #@param ["SAEHD", "Quick96"]

#@markdown <sup>**!python** main.py **train** **--training-data-src-dir** workspace/data_src/aligned **--training-data-dst-dir** workspace/data_dst/aligned **--pretraining-data-dir** pretrain **--model-dir** workspace/model **--model** " + $Model</sup>
cmd = "main.py train --training-data-src-dir workspace/data_src/aligned --training-data-dst-dir workspace/data_dst/aligned --pretraining-data-dir pretrain --model-dir workspace/model --model " + Model

!python $cmd

------------------------------------------------------------------
#**Convert frames**
------------------------------------------------------------------

In [ ]:
#@title Convert
Model = "SAEHD" #@param ["SAEHD", "Quick96"]

#@markdown <sup>**!python** "main.py **convert** **--input-dir** workspace/data_dst/aligned **--output-dir** workspace/data_dst/merged **--model-dir** workspace/model **--model** " + $Model</sup>
cmd = "main.py merge --input-dir workspace/data_dst --output-dir workspace/data_dst/merged --output-mask-dir workspace/data_dst/merged_mask --aligned-dir workspace/data_dst/aligned --model-dir workspace/model --model "+Model

!python $cmd

------------------------------------------------------------------
#**Get Final Result Video**
------------------------------------------------------------------

In [ ]:
!python main.py videoed video-from-sequence --input-dir workspace/data_dst/merged --output-file workspace/result.mp4 --reference-file workspace/data_dst.mp4

------------------------------------------------------------------
###If your session got **DISCONNECTED** or you want to **RESUME your work (or maybe you just switched to a different account)**, then execute the following lines of code.
------------------------------------------------------------------
####**Make sure to use the same google drive account which contains your previous work**

In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)
%cd "drive"
%cd "My Drive"
%cd "DFL-Colab-Shraeyas"
!pip install -r requirements-colab.txt

------------------------------------------------------------------
##If you clicked on **RESTART RUNTIME** in the above step, then execute the following lines.
------------------------------------------------------------------

In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)
%cd "drive"
%cd "My Drive"
%cd "DFL-Colab-Shraeyas"
!pip install -r requirements-colab.txt

###**After excuting these lines, simply continue with the step you were doing before.**